In [1]:
# Mount drive and unzip as before
from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/Comys_Hackathon5.zip"
!unzip -q "{zip_path}" -d /content/


Mounted at /content/drive


In [2]:
import os
import shutil

# Paths to your datasets
train_dir = "/content/Comys_Hackathon5/Task_B/train"
val_dir   = "/content/Comys_Hackathon5/Task_B/val"

# Get list of class folders
train_classes = set(os.listdir(train_dir))
val_classes   = set(os.listdir(val_dir))

# Find missing classes
missing_classes = train_classes - val_classes
print(f"Classes missing in val: {missing_classes}")

# Copy each missing class directory from train to val
for class_name in missing_classes:
    src_class_dir = os.path.join(train_dir, class_name)
    dest_class_dir = os.path.join(val_dir, class_name)

    # Copy entire directory tree
    shutil.copytree(src_class_dir, dest_class_dir)
    print(f"Copied {class_name} to val dataset.")

print("✅ All missing classes copied.")


Classes missing in val: {'Larry_Ralston', 'Mario_Alfaro-Lopez', 'Peter_Mugyeni', 'Tim_Norbeck', 'Sergei_Alexandrovitch_Ordzhonikidze', 'Kevin_Gil', 'Lindsay_Benko', 'Eva_Mendes', 'Larry_Ellison', 'Richard_Jewell', 'Sophia_Loren', 'Maurice_Strong', 'Connie_Chung', 'Yuri_Luzhkov', 'Sandra_Milo', 'Charles_Pickering', 'Richard_Rodriguez', 'Michael_Guiler', 'Brian_Schneider', 'Claudio_Abbado', 'Tom_Lantos', 'Bill_Kollar', 'Marian_Dolan', 'Takenori_Kanzaki', 'Eric_Ryan_Donnelly', '089_frontal', 'Emma_Nicholson', 'Peter_Fisher', 'Joseph_LePore', 'Kate_Starbird', 'Michael_Taylor', 'Paul_Li_Calsi', 'Carlos_Lordkipanitse', 'Jose_Lina', 'Kristin_Davis', '128_frontal', 'Dion_Glover', 'Valerie_Harper', 'Desmon_Farmer', 'Carlos_Salinas', 'John_Kerr', 'Michael_Sullivan', 'Brock_Berlin', 'Raoul_Ruiz', 'Mona_Ayoub', 'Daniel_Pearl', 'Carol_Burnett', 'Budd_Schulberg', 'John_Bolton', 'Douglas_Faneuil', 'Bryan_Chui', 'Faisal_Saleh_Hayat', 'Dick_Latessa', 'Tiger_Woods', 'Jerry_Sexton', 'Tyler_Grillo', 'Kell

In [3]:
# Imports
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [4]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),
    transforms.RandomApply([transforms.GaussianBlur(3)], p=0.3),
    transforms.RandomAffine(degrees=10, translate=(0.1,0.1)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225])
])

In [5]:
taskB_train_dir = "/content/Comys_Hackathon5/Task_B/train"
taskB_val_dir   = "/content/Comys_Hackathon5/Task_B/val"


In [6]:
# =======================================
taskB_train_ds = datasets.ImageFolder(taskB_train_dir, transform=transform_train)
taskB_val_ds   = datasets.ImageFolder(taskB_val_dir, transform=transform_val)

taskB_train_loader = DataLoader(taskB_train_ds, batch_size=32, shuffle=True, num_workers=2)
taskB_val_loader   = DataLoader(taskB_val_ds, batch_size=32, shuffle=False, num_workers=2)

num_classes_B = len(taskB_train_ds.classes)
print(f"Number of person identities: {num_classes_B}")

Number of person identities: 877


In [7]:
def create_model(num_classes):
    model = models.resnet18(weights='IMAGENET1K_V1')
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model.to(device)

model = create_model(num_classes_B)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 176MB/s]


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

def evaluate_model(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total

def train_model(model, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_acc = correct / total
        val_acc = evaluate_model(model, val_loader)
        print(f"[{epoch+1}/{num_epochs}] Train Loss: {running_loss/total:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

In [ ]:
print("\n🟢 Training Task B: Face Recognition")
train_model(model, taskB_train_loader, taskB_val_loader, num_epochs=50)


🟢 Training Task B: Face Recognition


Epoch 1/50: 100%|██████████| 482/482 [03:09<00:00,  2.54it/s]


[1/50] Train Loss: 5.4765 | Train Acc: 0.1774 | Val Acc: 0.0006


Epoch 2/50: 100%|██████████| 482/482 [03:05<00:00,  2.60it/s]


[2/50] Train Loss: 4.3170 | Train Acc: 0.2912 | Val Acc: 0.0009


...

Epoch 47/50: 100%|██████████| 482/482 [03:05<00:00,  2.59it/s]


[47/50] Train Loss: 0.3291 | Train Acc: 0.9261 | Val Acc: 0.0030


Epoch 48/50: 100%|██████████| 482/482 [03:07<00:00,  2.58it/s]


[48/50] Train Loss: 0.3017 | Train Acc: 0.9316 | Val Acc: 0.0032


Epoch 49/50: 100%|██████████| 482/482 [03:04<00:00,  2.61it/s]


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

def evaluate_f1_confusion(model, data_loader):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Classification report using integer labels
    print(classification_report(all_labels, all_preds, zero_division=0))

    # Confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(12,10))
    sns.heatmap(cm, annot=False, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    plt.title("Confusion Matrix")
    plt.show()

# ✅ Now call
evaluate_f1_confusion(model, taskB_val_loader)


In [ ]:
from sklearn.metrics import f1_score

def compute_macro_f1(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Macro-averaged F1-Score: {macro_f1:.4f}")

compute_macro_f1(model, taskB_val_loader)


In [ ]:
from sklearn.metrics import f1_score

def compute_macro_f1(model, data_loader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    print(f"Macro-averaged F1-Score: {macro_f1:.4f}")

compute_macro_f1(model, taskB_val_loader)


In [ ]:
val_acc = evaluate_model(model, taskB_val_loader)
print(f"Validation Accuracy: {val_acc:.4f}")


In [ ]:
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image

# Load your image
img_path = "/content/John_Nash_0001.jpg"  # adjust if needed
image = Image.open(img_path).convert("RGB")

# Apply the same transform as validation
input_tensor = transform_val(image).unsqueeze(0).to(device)

# Get predictions
model.eval()
with torch.no_grad():
    outputs = model(input_tensor)
    probs = F.softmax(outputs, dim=1).cpu().numpy()[0]

# Top 3 predictions
top3_idx = probs.argsort()[-3:][::-1]
top3_probs = probs[top3_idx]
top3_names = [taskB_train_ds.classes[i] for i in top3_idx]

# Print top 3
print("🔝 Top 3 predictions:")
for i in range(3):
    print(f"{top3_names[i]}: {top3_probs[i]:.4f}")

# Plot image with top prediction
plt.imshow(image)
plt.title(f"Predicted: {top3_names[0]} ({top3_probs[0]:.2f})")
plt.axis("off")
plt.show()


In [ ]:
# Save the model weights after training
torch.save(model.state_dict(), "taskB_model.pth")
print("✅ Model saved as taskB_model.pth")
